# Option 4: K-Fold Cross-Validation (RECOMMENDED)

## Problem
Single 80/20 split creates validation set (72 images) that doesn't represent test distribution.

**Evidence**: Baseline Val R²=0.6852 → Kaggle R²=0.51 (gap of -0.175!)

## Solution
**5-Fold Cross-Validation with Ensemble**
- Train 5 ResNet18 models (proven best architecture)
- Each fold uses different 80/20 split
- Ensemble predictions (average across 5 models)

## Why This Works
1. **Better validation estimate**: 5 different validation sets
2. **Ensemble effect**: Averaging reduces overfitting
3. **Proven architecture**: Using ResNet18, not experimental

## Expected
**Kaggle R²: 0.53-0.55** (improves from baseline 0.51)

## Time
- DEBUG: ~20 min (2 epochs × 5 folds)
- FULL: ~4 hours (45 epochs × 5 folds)

---

## Part 1: Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm.auto import tqdm
from datetime import datetime

sns.set_style('whitegrid')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✓ Setup complete")

In [ ]:
# ================================================================
# CONFIGURATION: 5-FOLD CROSS-VALIDATION
# ================================================================

DEBUG_MODE = False  # Set to False for full training

BATCH_SIZE = 16
TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
COMPETITION_WEIGHTS = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5])
NUM_FOLDS = 5

if DEBUG_MODE:
    print("\n" + "🐛"*40)
    print("DEBUG MODE: 2 epochs/phase per fold")
    print("🐛"*40 + "\n")

def get_epochs(full):
    return 2 if DEBUG_MODE else full

# Use ResNet18 (proven best) with standard config
BASE_CONFIG = {
    'phase1_epochs': get_epochs(15),
    'phase2_epochs': get_epochs(30),
    'phase1_lr': 3e-4,
    'phase2_lr': 3e-4,
    'phase2_backbone_lr': 1e-5,
    'weight_decay': 1e-4,
    'hidden_dim': 256,
    'dropout': 0.2,
    'backbone': 'resnet18',
    'use_scheduler': False
}

print("="*80)
print("K-FOLD CROSS-VALIDATION")
print("="*80)
print(f"\nFolds: {NUM_FOLDS}")
print(f"Model: ResNet18 (proven best)")
print(f"Phase 1: {BASE_CONFIG['phase1_epochs']} epochs")
print(f"Phase 2: {BASE_CONFIG['phase2_epochs']} epochs")
print(f"\n⏱️ {'DEBUG: ~20 min' if DEBUG_MODE else 'FULL: ~4 hours'}")
print("="*80)

In [ ]:
# Load full dataset
train_enriched = pd.read_csv('../../competition/train_enriched.csv')
train_enriched['Sampling_Date'] = pd.to_datetime(train_enriched['Sampling_Date'])
train_enriched['full_image_path'] = train_enriched['image_path'].apply(lambda x: f'../../competition/{x}')

print(f"Full dataset: {len(train_enriched)} images")
print(f"Will create {NUM_FOLDS} folds of ~{len(train_enriched)//NUM_FOLDS} val images each")

# Calculate normalization on FULL dataset (consistent across folds)
target_means = torch.tensor([train_enriched[col].mean() for col in TARGET_COLS], dtype=torch.float32)
target_stds = torch.tensor([train_enriched[col].std() for col in TARGET_COLS], dtype=torch.float32)

print("\nNormalization (full dataset):")
for i, col in enumerate(TARGET_COLS):
    print(f"  {col:15s}: mean={target_means[i]:.2f}g, std={target_stds[i]:.2f}g")

In [ ]:
# Prepare tabular feature names
weather_features = [
    'rainfall_7d', 'rainfall_30d',
    'temp_max_7d', 'temp_min_7d', 'temp_mean_7d', 'temp_mean_30d', 'temp_range_7d',
    'et0_7d', 'et0_30d',
    'water_balance_7d', 'water_balance_30d',
    'days_since_rain', 'daylength', 'season'
]

continuous_features = weather_features + ['Pre_GSHH_NDVI', 'Height_Ave_cm']

print(f"✓ Feature lists prepared")
print(f"  Weather features: {len(weather_features)}")
print(f"  Continuous features: {len(continuous_features)}")

## Part 2: Dataset Classes & Model

In [ ]:
class NormalizedDataset(Dataset):
    def __init__(self, dataframe, target_means, target_stds, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds

        transform_list = [transforms.Resize((224, 224))]
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.transform = transforms.Compose(transform_list)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        return {'image': img, 'targets': targets_normalized, 'targets_original': targets}

class MultimodalDataset(Dataset):
    def __init__(self, dataframe, target_means, target_stds, weather_features, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds
        self.weather_features = weather_features

        transform_list = [transforms.Resize((224, 224))]
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.transform = transforms.Compose(transform_list)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        weather = torch.tensor(row[self.weather_features].values.astype('float32'), dtype=torch.float32)
        ndvi = torch.tensor(row['Pre_GSHH_NDVI'], dtype=torch.float32)
        height = torch.tensor(row['Height_Ave_cm'], dtype=torch.float32)
        state = torch.tensor(row['State_encoded'], dtype=torch.long)
        species = torch.tensor(row['Species_encoded'], dtype=torch.long)
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        return {
            'image': img, 'weather': weather, 'ndvi': ndvi, 'height': height,
            'state': state, 'species': species,
            'targets': targets_normalized, 'targets_original': targets
        }

print("✓ Dataset classes defined")

In [ ]:
class AuxiliaryPretrainedModel(nn.Module):
    def __init__(self, num_outputs=5, hidden_dim=256, dropout=0.2, num_states=4, num_species=15):
        super().__init__()
        # ResNet18 backbone
        model = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(model.children())[:-1])
        feature_dim = 512

        # Auxiliary heads
        self.ndvi_head = nn.Linear(feature_dim, 1)
        self.height_head = nn.Linear(feature_dim, 1)
        self.weather_head = nn.Linear(feature_dim, 14)
        self.state_head = nn.Linear(feature_dim, num_states)
        self.species_head = nn.Linear(feature_dim, num_species)

        # Biomass head
        self.biomass_head = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_outputs)
        )

    def forward(self, x, mode='biomass'):
        features = self.backbone(x).flatten(1)
        if mode == 'auxiliary':
            return {
                'ndvi': self.ndvi_head(features),
                'height': self.height_head(features),
                'weather': self.weather_head(features),
                'state': self.state_head(features),
                'species': self.species_head(features)
            }
        else:
            return self.biomass_head(features)

print("✓ Model defined (ResNet18)")

## Part 3: Utility Functions

In [ ]:
def calculate_competition_r2(predictions, targets):
    per_target_r2 = []
    competition_r2 = 0
    weights = COMPETITION_WEIGHTS.numpy()
    for i in range(5):
        r2 = r2_score(targets[:, i], predictions[:, i])
        per_target_r2.append(r2)
        competition_r2 += weights[i] * r2
    return competition_r2, per_target_r2

def auxiliary_loss(predictions, targets):
    loss_ndvi = F.mse_loss(predictions['ndvi'].squeeze(), targets['ndvi'])
    loss_height = F.mse_loss(predictions['height'].squeeze(), targets['height'])
    loss_weather = F.mse_loss(predictions['weather'], targets['weather'])
    loss_state = F.cross_entropy(predictions['state'], targets['state'])
    loss_species = F.cross_entropy(predictions['species'], targets['species'])
    total_loss = (
        0.2 * loss_ndvi + 0.2 * loss_height + 0.3 * loss_weather +
        0.15 * loss_state + 0.15 * loss_species
    )
    return total_loss

print("✓ Utility functions defined")

## Part 4: Training Functions

In [ ]:
def train_phase1(model, train_loader, val_loader, num_epochs, lr, fold_name):
    print(f"\nPHASE 1: AUXILIARY PRETRAINING - {fold_name}")
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = {
                'ndvi': batch['ndvi'].to(device),
                'height': batch['height'].to(device),
                'weather': batch['weather'].to(device),
                'state': batch['state'].to(device),
                'species': batch['species'].to(device)
            }
            optimizer.zero_grad()
            predictions = model(images, mode='auxiliary')
            loss = auxiliary_loss(predictions, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}: Loss={train_loss:.4f}")

    print(f"✓ Phase 1 complete\n")

print("✓ train_phase1() defined")

In [ ]:
def train_phase2(model, train_loader, val_loader, config, target_means, target_stds, fold_name):
    print(f"PHASE 2: BIOMASS FINE-TUNING - {fold_name}")
    num_epochs = config['phase2_epochs']
    lr_head = config['phase2_lr']
    lr_backbone = config['phase2_backbone_lr']

    model = model.to(device)
    optimizer = torch.optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': lr_backbone},
        {'params': model.biomass_head.parameters(), 'lr': lr_head}
    ], weight_decay=config['weight_decay'])

    criterion = nn.MSELoss()
    best_r2 = -float('inf')

    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = batch['targets'].to(device)
            optimizer.zero_grad()
            outputs = model(images, mode='biomass')
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        train_loss /= len(train_loader.dataset)

        # Validate
        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets_original = batch['targets_original']
                outputs = model(images, mode='biomass')
                outputs_denorm = outputs.cpu() * target_stds + target_means
                all_preds.append(outputs_denorm.numpy())
                all_targets.append(targets_original.numpy())

        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        val_r2, _ = calculate_competition_r2(all_preds, all_targets)

        print(f"Epoch {epoch+1}/{num_epochs}: Loss={train_loss:.4f}, Val R²={val_r2:+.4f}")

        if val_r2 > best_r2:
            best_r2 = val_r2
            torch.save(model.state_dict(), f'model4b_{fold_name}_phase2_best.pth')
            print(f"  💾 New best: R²={best_r2:+.4f}")

    print(f"\n✓ Phase 2 complete! Best R²={best_r2:+.4f}\n")
    return best_r2

print("✓ train_phase2() defined")

---
## Part 5: K-Fold Training

Train 5 models, one for each fold.

In [ ]:
# Storage for all folds
fold_results = []
fold_models = []
fold_val_indices = []

# Create K-fold splits
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

print("="*80)
print("STARTING K-FOLD TRAINING")
print("="*80)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

In [ ]:
for fold_idx, (train_indices, val_indices) in enumerate(kf.split(train_enriched)):
    print(f"\n{'='*80}")
    print(f"FOLD {fold_idx+1}/{NUM_FOLDS}")
    print(f"{'='*80}")
    print(f"Train: {len(train_indices)} images, Val: {len(val_indices)} images")

    # Split data
    train_fold = train_enriched.iloc[train_indices].copy()
    val_fold = train_enriched.iloc[val_indices].copy()

    # Scale features for this fold
    scaler = StandardScaler()
    train_fold[continuous_features] = scaler.fit_transform(train_fold[continuous_features])
    val_fold[continuous_features] = scaler.transform(val_fold[continuous_features])

    # Encode categorical
    le_state = LabelEncoder()
    le_species = LabelEncoder()
    train_fold['State_encoded'] = le_state.fit_transform(train_fold['State'])
    train_fold['Species_encoded'] = le_species.fit_transform(train_fold['Species'])
    val_fold['State_encoded'] = le_state.transform(val_fold['State'])
    val_fold['Species_encoded'] = le_species.transform(val_fold['Species'])

    num_states = len(le_state.classes_)
    num_species = len(le_species.classes_)

    # Create datasets
    train_dataset_simple = NormalizedDataset(train_fold, target_means, target_stds, augment=True)
    train_dataset_multi = MultimodalDataset(train_fold, target_means, target_stds, weather_features, augment=True)
    val_dataset_simple = NormalizedDataset(val_fold, target_means, target_stds, augment=False)
    val_dataset_multi = MultimodalDataset(val_fold, target_means, target_stds, weather_features, augment=False)

    # Create model
    model = AuxiliaryPretrainedModel(
        num_outputs=5,
        hidden_dim=BASE_CONFIG['hidden_dim'],
        dropout=BASE_CONFIG['dropout'],
        num_states=num_states,
        num_species=num_species
    )

    # Create dataloaders
    train_loader_multi = DataLoader(train_dataset_multi, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader_multi = DataLoader(val_dataset_multi, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    train_loader_simple = DataLoader(train_dataset_simple, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader_simple = DataLoader(val_dataset_simple, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Phase 1
    train_phase1(
        model=model,
        train_loader=train_loader_multi,
        val_loader=val_loader_multi,
        num_epochs=BASE_CONFIG['phase1_epochs'],
        lr=BASE_CONFIG['phase1_lr'],
        fold_name=f'Fold{fold_idx+1}'
    )

    # Phase 2
    best_r2 = train_phase2(
        model=model,
        train_loader=train_loader_simple,
        val_loader=val_loader_simple,
        config=BASE_CONFIG,
        target_means=target_means,
        target_stds=target_stds,
        fold_name=f'Fold{fold_idx+1}'
    )

    # Load best checkpoint
    model.load_state_dict(torch.load(f'model4b_Fold{fold_idx+1}_phase2_best.pth'))
    model.eval()

    # Store
    fold_results.append({
        'fold': fold_idx + 1,
        'best_r2': best_r2
    })
    fold_models.append(model)
    fold_val_indices.append(val_indices)

    print(f"✅ Fold {fold_idx+1} complete! R²={best_r2:+.4f}")

print(f"\n{'='*80}")
print("K-FOLD TRAINING COMPLETE")
print(f"{'='*80}")
print(f"\nFold R² scores:")
for res in fold_results:
    print(f"  Fold {res['fold']}: R²={res['best_r2']:+.4f}")
mean_r2 = np.mean([r['best_r2'] for r in fold_results])
std_r2 = np.std([r['best_r2'] for r in fold_results])
print(f"\nMean: {mean_r2:+.4f}")
print(f"Std:  {std_r2:.4f}")

---
## Part 6: Ensemble Evaluation

Evaluate the ensemble of all 5 models.

In [ ]:
print("\n" + "="*80)
print("ENSEMBLE VALIDATION")
print("="*80)

# For each fold's validation set, get predictions from all 5 models
all_ensemble_preds = []
all_ensemble_targets = []

for fold_idx, val_indices in enumerate(fold_val_indices):
    val_fold = train_enriched.iloc[val_indices]
    val_dataset = NormalizedDataset(val_fold, target_means, target_stds, augment=False)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Get predictions from all models
    fold_preds = []
    for model in fold_models:
        model = model.to(device)
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                outputs = model(images, mode='biomass')
                outputs_denorm = outputs.cpu() * target_stds + target_means
                preds.append(outputs_denorm.numpy())
        fold_preds.append(np.vstack(preds))

    # Average predictions
    ensemble_preds = np.mean(fold_preds, axis=0)
    targets = val_fold[TARGET_COLS].values

    all_ensemble_preds.append(ensemble_preds)
    all_ensemble_targets.append(targets)

# Combine all folds
all_ensemble_preds = np.vstack(all_ensemble_preds)
all_ensemble_targets = np.vstack(all_ensemble_targets)

# Calculate ensemble R²
ensemble_r2, ensemble_per_target = calculate_competition_r2(all_ensemble_preds, all_ensemble_targets)

print(f"\n{'='*80}")
print("ENSEMBLE RESULTS")
print(f"{'='*80}")
print(f"\nEnsemble R² ({NUM_FOLDS} models averaged): {ensemble_r2:+.4f}")
print(f"Mean single model R²: {mean_r2:+.4f}")
print(f"Ensemble improvement: {ensemble_r2 - mean_r2:+.4f}")

baseline_r2 = 0.6852
baseline_kaggle = 0.51
print(f"\n📊 vs BASELINE:")
print(f"  Baseline (single ResNet18): Val={baseline_r2:+.4f}, Kaggle={baseline_kaggle:+.2f}")
print(f"  Ensemble ({NUM_FOLDS} models): Val={ensemble_r2:+.4f}")
print(f"  Val improvement: {ensemble_r2 - baseline_r2:+.4f}")

print(f"\nPer-target R² (Ensemble):")
for i, col in enumerate(TARGET_COLS):
    print(f"  {col:15s}: {ensemble_per_target[i]:+.4f}")

print(f"\n✅ Ensemble ready for Kaggle submission!")
print(f"   Expected Kaggle: ~0.53-0.55 (baseline was {baseline_kaggle:+.2f})")

---
## Summary

### Approach
5-fold cross-validation with ResNet18, ensemble predictions

### Results
[See output above]

### Key Insights
1. **Ensemble Effect**: Averaging 5 models typically improves ~0.01-0.02 R²
2. **Better Validation**: Multiple folds give more robust estimate
3. **Expected Kaggle**: If ensemble Val R² ≈ 0.685-0.69 → Kaggle ~0.53-0.55

### Next Steps
1. Create Kaggle submission using ensemble
2. Compare with Option 1 (EfficientNet tuning)
3. Use best performing approach for final submission